### prueba scrapear!

In [98]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent

In [100]:
ua = UserAgent()
header = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

#### carrefour

In [101]:
list_pages_carrefour = list(range(0,336,24))
table_pages_carrefour=[]
for number in list_pages_carrefour:
    url_carrefour = f'https://www.carrefour.es/supermercado/bebidas/cerveza/todas-las-cervezas/N-1uq8b5u/c?No={number}'
    html = requests.get(url_carrefour, headers=header).text
    soup_carrefour = BeautifulSoup(html, 'lxml')
    table_carrefour = soup_carrefour.find_all({'article':'product-card-item'})
    table_pages_carrefour.append(table_carrefour)

In [102]:
table_pages_carrefour[0][23]

<article class="product-card-item">
<div class="right-side">
<div class="photo">
<a class="js-gap-product-click-super" data-product-discount="" data-productid="526715428" data-special-campaign="" href="/supermercado/cerveza-alhambra-reserva-1925-pack-de-6-botellas-de-33-cl-alhambra/R-526715428/p" title="Cerveza Alhambra Reserva 1925 pack de 6 botellas de 33 cl.">
<img alt="Cerveza Alhambra Reserva 1925 pack de 6 botellas de 33 cl." aria-describedby="product-526715428" sizes="(max-width: 280px) 80px,(max-width: 490px) 100px, (max-width: 768px) 560px, (max-width: 991px) 280px, (max-width: 1280px) 560px" src="https://static.carrefour.es/hd_280x_/img_pim_food/272331_00_1.jpg" srcset="https://static.carrefour.es/hd_560x_/img_pim_food/272331_00_1.jpg 560w, https://static.carrefour.es/hd_280x_/img_pim_food/272331_00_1.jpg 280w, https://static.carrefour.es/hd_100x_/img_pim_food/272331_00_1.jpg 100w, https://static.carrefour.es/hd_100x_/img_pim_food/272331_00_1.jpg  80w" width="100%"/>
</a>
</d

In [103]:
len(table_pages_carrefour[0])

25

In [104]:
data_raw_carrefour=[]
for i in range(len(table_pages_carrefour)):
    for j in range(len(table_pages_carrefour[i])):
        rows_carrefour = table_pages_carrefour[i][j].find_all(['span','p','a'], {'class': ['price','price-less','format-price','js-gap-product-click-super']})
        promotion_carrefour = table_pages_carrefour[i][j].find_all(['p'], {'class': ['promocion-copy']})
        if len(rows_carrefour) != 0:
            if len(promotion_carrefour) != 0:
                data_raw_carrefour.append(rows_carrefour + promotion_carrefour)
            else:
                data_raw_carrefour.append(rows_carrefour)
                
            

In [105]:
data_raw_carrefour[20]

[<a class="js-gap-product-click-super" data-product-discount="" data-productid="520661354" data-special-campaign="qvpromo730001_feliz-aniversario|promof2320115_2a-al-50" href="/supermercado/cerveza-san-miguel-0-0-sin-alcohol-lager-pack-de-12-latas-de-33-cl-san-miguel-0-0-/R-520661354/p" title="Cerveza San Miguel 0,0 sin alcohol Lager pack de 12 latas de 33 cl.">
 <img alt="Cerveza San Miguel 0,0 sin alcohol Lager pack de 12 latas de 33 cl." aria-describedby="product-520661354" sizes="(max-width: 280px) 80px,(max-width: 490px) 100px, (max-width: 768px) 560px, (max-width: 991px) 280px, (max-width: 1280px) 560px" src="https://static.carrefour.es/hd_280x_/img_pim_food/646833_00_1.jpg" srcset="https://static.carrefour.es/hd_560x_/img_pim_food/646833_00_1.jpg 560w, https://static.carrefour.es/hd_280x_/img_pim_food/646833_00_1.jpg 280w, https://static.carrefour.es/hd_100x_/img_pim_food/646833_00_1.jpg 100w, https://static.carrefour.es/hd_100x_/img_pim_food/646833_00_1.jpg  80w" width="100%"/>

In [106]:
len(data_raw_carrefour)

332

In [107]:
#price in euros
# price_raw = data_raw[0][1].text

def get_price(data_text):
    price_raw = data_text.text
    price = re.sub('\xa0€','',price_raw)
    price = re.sub('\n','',price)
    return float(re.sub(",",'.',price))

In [108]:
#price per liter
# price_l_raw = data_raw[0][2].text

def get_price_l(data_text):
    price_l_raw = data_text.text
    price_l = re.findall('[0-9]+,[0-9]+', price_l_raw)
    price_liter = re.sub(',','.',price_l[0])
    return float(price_liter)

In [109]:
#title
# title_raw = data_raw[0][3].text

def get_title(data_text):
    title_raw = data_text.text
    title = re.sub("\n",'',title_raw)
    return title

In [110]:
#promotion product

def get_promotion(data_text):
        promotion_raw = data_text.text
        promotion = re.sub("\n",'',promotion_raw)
        return promotion


In [111]:
#brand
def get_brand(data_text):
    brand_raw = data_text.text
    brand = re.split('(Cerveza )|( pack)|( botella)|( lata)+|(cl)',brand_raw)
    return brand[6]

In [112]:
#container (carrefour  / alcampo)
def get_container(data_text):
    container_raw = data_text.text
    container = re.findall('botella|lata|barril',container_raw)
    if len(container)!=0:
        return container[0]
    else:
        return 'No specified'

In [113]:
#volumen
def get_volumen_unid(data_text):
    volumen_raw = data_text.text 
    volumen = re.findall('[0-9]+ cl',volumen_raw)
    if len(volumen)!=0:
        return volumen[0]
    else:
        volumen_l = re.findall('[0-9]+[ mcl]+',volumen_raw)
        return volumen_l[0]

In [114]:
#quantity (carrefour  / alcampo)
def get_quantity_pack(data_text):
    quantity_raw = data_text.text
    quantity_split = re.split('pack de|pack',quantity_raw)
    if len(quantity_split)==2:
        quantity = re.findall('[0-9]+',quantity_split[1])
        return quantity[0]
    else:
        return 1

In [115]:
#image
def get_image(data_text):
    return data_text.find_all('img')[0].get('src')

In [116]:
data_beer_carrefour = pd.DataFrame(index=range(0,len(data_raw_carrefour)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_carrefour)):
    data_beer_carrefour.iloc[beer_number,0] = get_price(data_raw_carrefour[beer_number][1])
    data_beer_carrefour.iloc[beer_number,1] = get_price_l(data_raw_carrefour[beer_number][2])
    data_beer_carrefour.iloc[beer_number,2] = get_title(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,4] = get_brand(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,5] = get_container(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,6] = get_volumen_unid(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,7] = get_quantity_pack(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,8] = get_image(data_raw_carrefour[beer_number][0])
    data_beer_carrefour.iloc[beer_number,9] = 'Carrefour'
    if len(data_raw_carrefour[beer_number]) > 4:
        data_beer_carrefour.iloc[beer_number,3] = get_promotion(data_raw_carrefour[beer_number][4])
    else:
        data_beer_carrefour.iloc[beer_number,3] = 'No promotion'


In [117]:
data_beer_carrefour.head()

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket
0,11.21,1.89,Cerveza Heineken Lager pack de 18 latas de 33 cl.,No promotion,Heineken Lager,lata,33 cl,18,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
1,14.8,1.6,Cerveza Cruzcampo Pilsen pack de 28 latas de 3...,No promotion,Cruzcampo Pilsen,lata,33 cl,28,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
2,5.15,1.56,Cerveza El Aguila pack de 10 latas de 33 cl.,No promotion,El Aguila,lata,33 cl,10,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
3,0.25,0.76,Cerveza Carrefour Holandesa lata 33 cl.,No promotion,Carrefour Holandesa,lata,33 cl,1,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
4,14.15,1.53,Cerveza Mahou Clásica pack de 28 latas de 33 cl.,No promotion,Mahou Clásica,lata,33 cl,28,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour


In [118]:
sum(data_beer_carrefour['price'])

901.4899999999999

## Alcampo

In [119]:
list_pages_alcampo = list(range(0,10))
table_pages_alcampo=[]
for number in list_pages_alcampo:
    url_alcampo = f'https://www.alcampo.es/compra-online/bebidas/cervezas/c/W1107?q=%3Arelevance&page={number}'
    html = requests.get(url_alcampo, headers=header).text
    soup_alcampo = BeautifulSoup(html, 'lxml')
    table_alcampo = soup_alcampo.find_all('div',{'class':'productGridItem'})
    table_pages_alcampo.append(table_alcampo)
    

In [120]:
len(table_pages_alcampo[0])

48

In [121]:
len(table_pages_alcampo)

10

In [122]:
data_raw_alcampo=[]
for i in range(len(table_pages_alcampo)):
    for j in range(len(table_pages_alcampo[i])):
        rows_alcampo = table_pages_alcampo[i][j].find_all(['span','div'],{'class': ['price',
                                                                                    'productName',
                                                                                    'thumb cut-alt-img']})
        promotion_alcampo = table_pages_alcampo[i][j].find_all('div', {'class': 'financiacionMensual'})
        if len(rows_alcampo) != 0:
            if len(promotion_alcampo) != 0:
                data_raw_alcampo.append(rows_alcampo + promotion_alcampo)
            else:
                data_raw_alcampo.append(rows_alcampo)
                

In [123]:
len(data_raw_alcampo)

432

In [217]:
#price alcampo / corte ingles / Dia
def get_price_alcampo(data_text):
    price_raw = data_text.text
    price = re.findall('[0-9,]+',price_raw)
    return float(re.sub(",",'.',price[0]))


In [218]:
#price_liter alcampo / corte ingles / Dia
def get_price_liter_alcampo(data_text):
    price_raw = data_text.text
    price = re.findall('[0-9,]+',price_raw)
    if len(price)==1:
        return float(re.sub(",",'.',price[0]))
    else:
        return float(re.sub(",",'.',price[1]))


In [219]:
#title alcampo / corte ingles
def get_tittle_alcampo(data_text):
    tittle_raw = data_text.text
    tittle = re.sub('\n','',tittle_raw)
    return tittle


In [220]:
#promotion alcampo
def get_promotion_alcampo(data_text):
    promotion_raw = data_text.text
    promotion = re.sub('\r\n\r\n\t\t\t\t\t\t\t\t\t\t','',promotion_raw)
    return promotion

In [221]:
#brand alcampo
def get_brand_alcampo(data_text):
    brand_raw = data_text.text
    brand = re.sub('\n','',brand_raw)
    brand_2 = re.findall('[A-Z 0-9,À-ÿ]{3,} ',brand)
    return brand_2[0]

In [322]:
#volumen alcampo
def get_volumen_unid_alcampo(data_text):
    volumen_raw = data_text.text 
    volumen = re.findall('[0-9.,]+ cl',volumen_raw)
    if len(volumen)!=0:
        return volumen[0]
    else:
        volumen_l = re.findall('[0-9.,]+[ mclL]+',volumen_raw)
        if len(volumen_l)==2:
            return volumen_l[1]
        else:
            return volumen_l[0]

In [323]:
#image alcampo
def get_image_alcampo(data_text):
    return data_text.find_all('img')[0].get('data-blzsrc')

In [324]:
data_raw_alcampo[0][1]

<div class="productName truncate">
<!-- Inicio UAT-203 -->
<span>Cerveza Clásica MAHOU pack de 28 latas de 33 cl.</span>
<!-- Fin UAT-203 -->
</div>

In [325]:
data_beer_alcampo= pd.DataFrame(index=range(0,len(data_raw_alcampo)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_alcampo)):
    data_beer_alcampo.iloc[beer_number,0] = get_price_alcampo(data_raw_alcampo[beer_number][2])
    data_beer_alcampo.iloc[beer_number,1] = get_price_liter_alcampo(data_raw_alcampo[beer_number][2])
    data_beer_alcampo.iloc[beer_number,2] = get_tittle_alcampo(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,4] = get_brand_alcampo(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,5] = get_container(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,6] = get_volumen_unid_alcampo(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,7] = get_quantity_pack(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,8] = get_image_alcampo(data_raw_alcampo[beer_number][0])
    data_beer_alcampo.iloc[beer_number,9] = 'Alcampo'
    if len(data_raw_alcampo[beer_number]) > 3:
        data_beer_alcampo.iloc[beer_number,3] = get_promotion_alcampo(data_raw_alcampo[beer_number][3])
    else:
        data_beer_alcampo.iloc[beer_number,3] = 'No promotion'

In [326]:
data_beer_alcampo.head(50)

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket
0,11.99,1.29,Cerveza Clásica MAHOU pack de 28 latas de 33 cl.,oferta válida hasta el 14/10/2020,MAHOU,lata,33 cl,28,https://www.alcampo.es/media/ha9/h39/100335357...,Alcampo
1,13.49,1.45,Cervezas MAHOU 5 ESTRELLAS pack 28 latas x 33 cl.,No promotion,MAHOU 5 ESTRELLAS,lata,33 cl,28,https://www.alcampo.es/media/h11/h90/902116140...,Alcampo
2,6.66,1.68,Cerveza HEINEKEN pack de 12 latas de 33 centi...,No promotion,HEINEKEN,lata,33 c,12,https://www.alcampo.es/media/h67/h4f/954684103...,Alcampo
3,5.28,1.33,"Cervezas AMSTEL 100 % MALTA pack 12 uds, de 3...",No promotion,AMSTEL 100,No specified,33 cl,12,https://www.alcampo.es/media/hc9/h32/104726094...,Alcampo
4,5.62,1.41,Cervezas Clásica MAHOU pack de 12 latas de 33...,No promotion,MAHOU,lata,33 c,12,https://www.alcampo.es/media/h64/hc7/965932407...,Alcampo
5,6.63,1.67,Cervezas MAHOU 5 ESTRELLAS pack de 12 latas de...,No promotion,MAHOU 5 ESTRELLAS,lata,33 cl,12,https://www.alcampo.es/media/h77/hbd/902116707...,Alcampo
6,9.11,1.51,Cervezas CRUZCAMPO pack 24 uds de 25 cl.,No promotion,CRUZCAMPO,No specified,25 cl,24,https://www.alcampo.es/media/h4a/h37/954688009...,Alcampo
7,5.99,1.51,Cerveza con limón Radler AMSTEL pack 12 uds. x...,No promotion,AMSTEL,No specified,33 cl,12,https://www.alcampo.es/media/hae/h18/104074999...,Alcampo
8,5.99,1.51,Cervezas CRUZCAMPO pack 12 uds x 33 cl.,No promotion,CRUZCAMPO,No specified,33 cl,12,https://www.alcampo.es/media/hfc/h51/104073672...,Alcampo
9,0.65,1.3,Cerveza ALHAMBRA PREMIUM LAGER lata de 50 cent...,No promotion,ALHAMBRA PREMIUM LAGER,lata,50 c,1,https://www.alcampo.es/media/h3b/hd5/104447574...,Alcampo


## El corte ingles

In [175]:
list_pages_corteingles = list(range(1,26))
table_pages_corteingles =[]
for number in list_pages_corteingles:
    url_corteingles = f'https://elcorteingles.es/supermercado/bebidas/cervezas/{number}/'
    html = requests.get(url_corteingles, headers=header).text
    soup_corteingles= BeautifulSoup(html, 'lxml')
    table_corteingles = soup_corteingles.find_all('div',{'class':'grid-item'})
    table_pages_corteingles.append(table_corteingles)

In [176]:
len(table_pages_corteingles)

26

In [179]:
len(table_pages_corteingles[25])

7

In [201]:
data_raw_corteingles=[]
for i in range(len(table_pages_corteingles)):
    for j in range(len(table_pages_corteingles[i])):
        rows_corteingles = table_pages_corteingles[i][j].find_all(['div','h3'],{'class': ['prices-price',
                                                                                    'product_tile-description',
                                                                                    'product_tile-image _fade']})
        promotion_corteingles = table_pages_corteingles[i][j].find_all('div', {'class': 'product_tile-offer offer'})
        if len(rows_corteingles) > 3:
            if len(promotion_corteingles) != 0:
                data_raw_corteingles.append(rows_corteingles + promotion_corteingles)
            else:
                data_raw_corteingles.append(rows_corteingles)
                

In [202]:
len(data_raw_corteingles)

455

In [239]:
#brand corte ingles
def get_brand_corteingles(data_text):
    brand_raw = data_text.text
    brand = re.split('cervez',brand_raw)
    return brand[0]

In [ ]:
#volumen corte ingles
def get_volumen_corteingles(data_text):
    brand_raw = data_text.text
    brand = re.split('botel',brand_raw)
    return brand[0]

In [259]:
#image corte ingles
def get_image_corteingles(data_text):
    return 'https:'+data_text.find_all('img')[0].get('src')

In [264]:
data_raw_corteingles[2][3].text

'MAHOU CLASICA cerveza rubia original pack 12 botellas 25 cl '

In [327]:
data_beer_corteingles = pd.DataFrame(index=range(0,len(data_raw_corteingles)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_corteingles)):
    data_beer_corteingles.iloc[beer_number,0] = get_price_alcampo(data_raw_corteingles[beer_number][1])
    data_beer_corteingles.iloc[beer_number,1] = get_price_liter_alcampo(data_raw_corteingles[beer_number][2])
    data_beer_corteingles.iloc[beer_number,2] = get_tittle_alcampo(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,4] = get_brand_corteingles(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,5] = get_container(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,6] = get_volumen_unid_alcampo(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,7] = get_quantity_pack(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,8] = get_image_corteingles(data_raw_corteingles[beer_number][0])
    data_beer_corteingles.iloc[beer_number,9] = 'El Corte Ingles'
    if len(data_raw_corteingles[beer_number]) > 4:
        data_beer_corteingles.iloc[beer_number,3] = get_promotion_alcampo(data_raw_corteingles[beer_number][4])
        if len(data_raw_corteingles[beer_number]) > 5:
            data_beer_corteingles.iloc[beer_number,3] += '..SECOND OFFERT: '+ get_promotion_alcampo(data_raw_corteingles[beer_number][5])
    else:
        data_beer_corteingles.iloc[beer_number,3] = 'No promotion'    

In [328]:
data_beer_corteingles.head()

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket
0,6.72,1.7,MAHOU CLASICA cerveza rubia original pack 12 l...,21% de descuento por compras superiores a 20 €...,MAHOU CLASICA,lata,33 cl,12,https://cdn.grupoelcorteingles.es/SGFM/dctm/ME...,El Corte Ingles
1,14.84,1.61,MAHOU CLASICA cerveza rubia original pack 28 l...,2ª-50%2ª unidad al 50% de descuentoPuedes comb...,MAHOU CLASICA,lata,33 cl,28,https://cdn.grupoelcorteingles.es/SGFM/dctm/ME...,El Corte Ingles
2,5,1.67,MAHOU CLASICA cerveza rubia original pack 12 b...,2ª-50%2ª unidad al 50% de descuentoPuedes comb...,MAHOU CLASICA,botella,25 cl,12,https://cdn.grupoelcorteingles.es/SGFM/dctm/ME...,El Corte Ingles
3,1.37,1.37,MAHOU CLASICA cerveza rubia original botella 1 l,21% de descuento por compras superiores a 20 €...,MAHOU CLASICA,botella,1 l,1,https://cdn.grupoelcorteingles.es/SGFM/dctm/ME...,El Corte Ingles
4,2.68,1.79,MAHOU CLASICA cerveza rubia original pack 6 bo...,21% de descuento por compras superiores a 20 €...,MAHOU CLASICA,botella,25 cl,6,https://cdn.grupoelcorteingles.es/SGFM/dctm/ME...,El Corte Ingles


## DIA

In [271]:
list_pages_dia= list(range(0,3))
table_pages_dia =[]
for number in list_pages_dia:
    url_dia = f'https://www.dia.es/compra-online/productos/bebidas/cervezas/c/WEB.008.064.00000?page={number}&disp='
    html = requests.get(url_dia, headers=header).text
    soup_dia= BeautifulSoup(html, 'lxml')
    table_dia = soup_dia.find_all('div',{'class':'prod_grid'})
    table_pages_dia.append(table_dia)

In [272]:
len(table_pages_dia)

3

In [273]:
len(table_pages_dia[2])

20

In [295]:
data_raw_dia=[]
for i in range(len(table_pages_dia)):
    for j in range(len(table_pages_dia[i])):
        rows_dia = table_pages_dia[i][j].find_all(['div','span'],{'class': ['price_container','details',
                                                                            'thumb']})
        if len(rows_dia) > 2:
            data_raw_dia.append(rows_dia)
                

In [296]:
len(data_raw_dia)

67

In [306]:
#title Dia
def get_tittle_dia(data_text):
    tittle_raw = data_text.text
    tittle = re.sub('\r\n\t\t\t\t','',tittle_raw)
    return tittle

In [312]:
#brand Dia
def get_brand_dia(data_text):
    brand_raw = data_text.text
    brand_sub = re.sub('\r\n\t\t\t\t','',brand_raw)
    brand = re.split('cervez',brand_sub)
    return brand[0]

In [338]:
#image dia
def get_image_dia(data_text):
    return data_text.find_all('img')[0].get('data-original')

In [344]:
data_raw_dia[24][2].text

'\n\r\n\t\t\t\t\t\r\n\t\t\t\t\t0,43\xa0€\n(1,30\xa0€/l.)\n'

In [345]:
data_beer_dia= pd.DataFrame(index=range(0,len(data_raw_dia)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_dia)):
    data_beer_dia.iloc[beer_number,0] = get_price_alcampo(data_raw_dia[beer_number][2])
    data_beer_dia.iloc[beer_number,1] = get_price_liter_alcampo(data_raw_dia[beer_number][2])
    data_beer_dia.iloc[beer_number,2] = get_tittle_dia(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,4] = get_brand_dia(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,5] = get_container(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,6] = get_volumen_unid_alcampo(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,7] = get_quantity_pack(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,8] = get_image_dia(data_raw_dia[beer_number][0])
    data_beer_dia.iloc[beer_number,9] = 'Dia'
    if len(data_raw_dia[beer_number]) > 3:
        data_beer_dia.iloc[beer_number,3] = get_promotion_alcampo(data_raw_dia[beer_number][3])
    else:
        data_beer_dia.iloc[beer_number,3] = 'No promotion' 

In [347]:
data_beer_dia.head()

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket
0,0.56,1.7,"SAN MIGUEL cerveza 0,0% alcohol lata 33 cl",No promotion,SAN MIGUEL,lata,33 cl,1,https://s1.dia.es/medias/h6e/h4e/9846950264862...,Dia
1,0.35,1.06,DIA cerveza especial lata 33 cl,No promotion,DIA,lata,33 cl,1,https://s2.dia.es/medias/h46/hb5/8822550659102...,Dia
2,5.89,2.97,1906 cerveza reserva especial pack 6 botellas ...,No promotion,1906,botella,33 cl,6,https://s2.dia.es/medias/h65/h32/9622060400670...,Dia
3,3.99,2.66,HEINEKEN cerveza pack 6 botellas 25 cl,No promotion,HEINEKEN,botella,25 cl,6,https://s2.dia.es/medias/ha3/h75/9847216668702...,Dia
4,6.47,1.63,AMSTEL cerveza pack 12 latas 33 cl,No promotion,AMSTEL,lata,33 cl,12,https://s0.dia.es/medias/ha2/hbf/9847081402398...,Dia


## Eroski

In [395]:
table_pages_eroski =[]
url_eroski = f'https://supermercado.eroski.es/es/supermercado/2060211-bebidas/2060233-cervezas/'
html = requests.get(url_eroski, headers=header).text
soup_eroski = BeautifulSoup(html, 'lxml')
table_eroski = soup_eroski.find_all('ul',{'class':'product_grid__grid__1k39q'})
table_pages_eroski.append(table_eroski)

In [396]:
len(table_pages_eroski[0])

0

## Amazon prime now

In [407]:
list_pages_amazon = list(range(1,10))
table_pages_amazon =[]
for number in list_pages_amazon:
    url_amazon = f'https://primenow.amazon.es/search?srs=6347717031&qid=1601728536&page={number}&ref=sr_pg_{number}&ie=UTF8'
    html = requests.get(url_amazon).text
    soup_amazon= BeautifulSoup(html, 'lxml')
    table_amazon = soup_amazon.find_all('main',{'class':'page_chrome__main__wEgLS'})
    table_pages_amazon.append(table_amazon)


In [408]:
len(table_pages_amazon)

9

In [409]:
len(table_pages_amazon[0])

0

In [394]:
soup_amazon

<!DOCTYPE html>
<html>
<head>
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<script type="text/javascript">
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function(b){for(var a;a=c.shift();)b(a[0],a[1],a[2])};b[a].isStub=1}};e.exec=function(b,a){return function(){try{return b.apply(this,arguments)}catch(c){ueLogError(c,{attribution:a||"undefined",logLevel:"WARN"})}}}})(ue_csm);


    var ue_err_chan = 'jserr-rw';
(function(d,e){function h(f,b){if(!(a.ec>a.mxe)&&f){a.ter.push(f);b=b||{};var c=f.logLevel||b.logLevel;c&&c!==k&&c!==m&&c!==n&&c!==p||a.ec++;c&&c!=k||a.ecf++;b.pageURL=""+(e.location?e.location.href:"");b.logLevel=c;b.attribution=f.attribut